
# Motor Imagery Left/Right Classification Using Machine Learning
## BCI-4-ALS, Ben Gurion University, 2020
## Evyatar Luvaton, Noam Siegel

In [1]:
import mne
import os
import numpy as np
import pyxdf
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [5]:
filename = "EEG_clean.csv"
df = pd.read_csv(filename)
df


,time,C03,C04,P07,P08,O01,O02,F07,F08,F03,F04,T07,T08,P03
0,58743.695935,9.890755e-12,-2.817302e-12,2.559730e-12,1.888267e-12,-7.140954e-12,-1.065814e-13,8.650858e-13,-3.402612e-12,2.376765e-12,-3.758771e-12,-3.151257e-12,-1.239897e-12,8.203216e-12
1,58743.704347,1.166108e+00,-5.561453e-01,-4.864093e+01,-2.637478e+01,-5.383466e+01,-3.564287e+01,-3.505946e+01,7.336168e+01,-5.021892e+01,-1.315151e+01,-4.175785e+01,8.230355e+01,-3.703367e+01
2,58743.712759,1.018287e+01,1.470553e+01,-4.398977e+01,-1.474130e+01,-5.845574e+01,-3.510867e+01,-3.464476e+01,8.121837e+01,-4.759550e+01,-5.367630e+00,-3.758946e+01,8.722343e+01,-3.656164e+01
3,58743.721170,1.719192e+01,2.940293e+01,-1.772598e+01,4.251417e+00,-3.756410e+01,-1.233011e+01,-1.458759e+01,5.718628e+01,-1.856070e+01,1.362155e+01,-1.193679e+01,5.320649e+01,-1.469571e+01
4,58743.729582,1.043654e+01,2.222838e+01,-2.380217e+01,-1.273932e+01,-3.899671e+01,-7.140335e+00,-1.372616e+01,6.147038e+01,-1.629825e+01,1.891058e+01,-1.067292e+01,5.354244e+01,-1.085520e+01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105545,59631.526792,2.028565e+01,2.251182e+01,-1.042487e+01,6.719527e+00,-1.115906e+01,2.683718e+00,-1.867360e+01,1.692279e+01,-2.363355e+01,2.380687e+01,-4.860197e+00,3.486146e+01,1.752819e+01
105546,59631.535204,1.652386e+01,1.833573e+01,-9.036746e+00,-1.193416e+00,-1.053629e+01,6.801212e+00,-1.510070e+01,1.650522e+01,-2.288107e+01,2.213111e+01,-2.589521e+00,3.065247e+01,1.623849e+01
105547,59631.543616,8.516816e+00,1.160051e+01,-2.016478e+01,-5.703144e+00,-2.324568e+01,-4.126666e+00,-3.097336e+01,5.349679e+00,-5.217469e+01,1.848406e+01,-1.680307e+01,4.293041e+01,9.461861e+00
105548,59631.552028,2.260067e+00,5.428247e+00,-2.109917e+01,-4.249676e+00,-2.436509e+01,-1.043571e+01,-3.195640e+01,-2.303871e+00,-5.447132e+01,1.139397e+01,-2.075299e+01,3.914296e+01,3.177659e+00


In [3]:
vec = np.genfromtxt("stimulus_vectors.csv")
vec

array([2., 2., 1., 0., 1., 2., 2., 0., 1., 2., 1., 2., 1., 1., 2., 1., 1.,
       0., 2., 2., 1., 0., 2., 0., 0., 1., 1., 1., 2., 0., 2., 1., 2., 1.,
       2., 2., 1., 0., 0., 1., 1., 2., 2., 2., 0., 0., 2., 2., 0., 2., 1.,
       0., 1., 0., 0., 1., 1., 2., 0., 0., 1., 0., 0., 0., 0., 1., 2., 1.,
       1., 2., 0., 0., 2., 2., 0., 0., 0., 0., 2., 1., 2., 0., 2., 2., 2.,
       1., 0., 0., 0., 2., 1., 0., 2., 2., 0., 2., 0., 1., 2., 1., 1., 1.,
       2., 1., 1., 2., 2., 2., 2., 2., 0., 0., 2., 0., 1., 1., 2., 2., 0.,
       2.])

In [4]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_filt-0-40_raw.fif')
raw = mne.io.read_raw_fif(sample_data_raw_file)


Opening raw data file C:\Users\noam\mne_data\MNE-sample-data\MEG\sample\sample_audvis_filt-0-40_raw.fif...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
        Average EEG reference (1 x 60)  idle
    Range : 6450 ... 48149 =     42.956 ...   320.665 secs
Ready.
